# TCN Architecture Analysis (Execution-Ready, Updated 2026-02-18)

This notebook is aligned with the current TAPE environment updates:
- drawdown lambda carry-over + decay
- rebalanced penalties with penalty budget cap
- intra-step TAPE delta shaping (rolling potential difference)
- step-level Sharpe checkpointing
- updated episode-length curriculum schedule


## 1) Setup and Imports


In [ ]:
#%cd /content/adaptive_portfolio_rl
#!ls -la data
#!find . -type d -name "__pycache__" | head

In [ ]:
#%cd /content/adaptive_portfolio_rl
#!rm -f data/daily_ohlcv_assets.csv
#!rm -f data/master_features_NORMALIZED.csv
#!rm -f data/processed_daily_macro_features.csv
#!find . -type d -name "__pycache__" -prune -exec rm -rf {} +
#!ls -la data

In [ ]:
# Cell A: sync repo
import os
if not os.path.exists("/content/adaptive_portfolio_rl/.git"):
    !git clone https://github.com/Dave-DKings/tape_tcn_project.git /content/adaptive_portfolio_rl
%cd /content/adaptive_portfolio_rl
!git fetch origin
!git reset --hard origin/main

/content/adaptive_portfolio_rl
HEAD is now at 79203cd Add execution-inertia smoothing and beta curriculum for turnover control


In [1]:
%cd /content/adaptive_portfolio_rl
!git fetch origin
!git reset --hard origin/main
!git rev-parse --short HEAD

/content/adaptive_portfolio_rl
HEAD is now at 79203cd Add execution-inertia smoothing and beta curriculum for turnover control
79203cd


In [ ]:
%cd /content/adaptive_portfolio_rl
!rm -f data/daily_ohlcv_assets.csv
!rm -f data/processed_daily_macro_features.csv
!rm -f data/master_features_NORMALIZED.csv
!rm -rf data/fundamentals
!find . -type d -name "__pycache__" -prune -exec rm -rf {} +

In [ ]:
# where am I
!pwd
!ls -la

In [5]:
# find results folders
!find . -maxdepth 3 -type d \( -name "tcn_fusion_results" -o -name "tcn_results" -o -name "output_logs" \) -print

In [6]:
# list files inside results
!find ./tcn_fusion_results -maxdepth 3 -type f | head -n 100

find: ‘./tcn_fusion_results’: No such file or directory


In [7]:
# delete old training outputs
!rm -rf tcn_fusion_results
#!rm -rf tcn_results
#!rm -rf results

In [ ]:
# Cell B: install project requirements (Colab-safe pinned stack)
%pip install -q -r requirements.txt


In [ ]:
#%pip install -q jedi==0.19.2

In [ ]:
%cd /content/adaptive_portfolio_rl
!zip -r tcn_artifacts.zip tcn_fusion_results tcn_results output_logs data
from google.colab import files
files.download("tcn_artifacts.zip")

In [ ]:
import numpy as np, pandas as pd, tensorflow as tf
import pandas_ta_classic as ta_classic
print('Versions:', np.__version__, pd.__version__, tf.__version__)
print('TF GPUs visible:', tf.config.list_physical_devices('GPU'))
!pip check

In [8]:
# ============================================================================
# SETUP: PROJECT ROOT, IMPORTS, REPRODUCIBILITY
# ============================================================================
import os
import sys
import json
import warnings
import importlib
from copy import deepcopy
from pathlib import Path
from datetime import datetime
import inspect

# Resolve project root robustly
project_root = Path.cwd()
if project_root.name != 'adaptive_portfolio_rl':
    if (project_root / 'adaptive_portfolio_rl').exists():
        project_root = project_root / 'adaptive_portfolio_rl'
    elif (project_root.parent / 'adaptive_portfolio_rl').exists():
        project_root = project_root.parent / 'adaptive_portfolio_rl'

# Ensure imports resolve to this project only
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

# Scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tensorflow as tf

# IMPORTANT: force-reload local modules so config edits take effect without stale state
import src.config as config_module
import src.data_utils as data_utils_module
import src.notebook_helpers.tcn_phase1 as tcn_phase1_module
import src.environment_tape_rl as env_mod
from src.environment_tape_rl import PortfolioEnvTAPE
importlib.reload(config_module)
importlib.reload(data_utils_module)
importlib.reload(tcn_phase1_module)
importlib.reload(env_mod)
#importlib.reload(phase_mod)

# Project imports (from freshly reloaded modules)
from src.data_utils import DataProcessor
from src.config import get_active_config, PROFILE_BALANCED_GROWTH, ASSET_TICKERS
from src.reproducibility_helper import set_all_seeds
from src.csv_logger import CSVLogger
from src.notebook_helpers.tcn_phase1 import (
    identify_covariance_columns,
    Phase1Dataset,
    run_experiment6_tape,
    evaluate_experiment6_checkpoint,
    create_experiment6_result_stub,
    load_training_metadata_into_config,
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

RANDOM_SEED = 42
set_all_seeds(RANDOM_SEED, deterministic=True)

import logging
logging.getLogger("src.environment_tape_rl").setLevel(logging.WARNING)

gpus = tf.config.list_physical_devices("GPU")
print("TF GPUs:", gpus)
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print('✅ Setup complete')
print('Project root:', project_root)
print('Config module path:', config_module.__file__)
print('Active fetch range from module:', config_module.DATA_FETCH_START_DATE, '->', config_module.DATA_FETCH_END_DATE)
print('Active analysis range from module:', config_module.ANALYSIS_START_DATE, '->', config_module.ANALYSIS_END_DATE)
print('Train split end from module:', config_module.TRAIN_TEST_SPLIT_DATE)
print('TensorFlow:', tf.__version__)
print('NumPy:', np.__version__)
print('Pandas:', pd.__version__)
print("action_execution_beta" in inspect.signature(PortfolioEnvTAPE.__init__).parameters)



🎲 Setting all random seeds to: 42
   ✅ Deterministic mode enabled (slower but reproducible)
   ✅ Python random seed set
   ✅ NumPy random seed set
   ✅ TensorFlow seed set
   ✅ Custom PPO agents seeded
TF GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Setup complete
Project root: /content/adaptive_portfolio_rl
Config module path: /content/adaptive_portfolio_rl/src/config.py
Active fetch range from module: 2003-09-02 -> 2025-09-01
Active analysis range from module: 2003-09-02 -> 2025-09-01
Train split end from module: 2021-09-01
TensorFlow: 2.19.0
NumPy: 2.0.2
Pandas: 2.2.2
True


## 2) Config and Run Controls


In [9]:
# ============================================================================
# LOAD ACTIVE CONFIG + CURRENT CONTROL SNAPSHOT
# ============================================================================
config = get_active_config('phase1')

# Force a single source-of-truth date window for this notebook run
# (keeps paper training split, extends test window to ~3 years)
config['DATA_FETCH_START_DATE'] = '2003-09-02'
config['DATA_FETCH_END_DATE'] = '2025-09-01'
config['ANALYSIS_START_DATE'] = '2003-09-02'
config['ANALYSIS_END_DATE'] = '2025-09-01'
config['TRAIN_TEST_SPLIT_DATE'] = '2021-09-01'

print('Forced notebook date window:', config['DATA_FETCH_START_DATE'], '->', config['DATA_FETCH_END_DATE'])
print('Forced analysis window:', config['ANALYSIS_START_DATE'], '->', config['ANALYSIS_END_DATE'])
print('Forced train split end:', config['TRAIN_TEST_SPLIT_DATE'])

# Keep defaults from config unless explicitly changed below
config['agent_params']['actor_critic_type'] = 'TCN'
config['agent_params']['evaluation_mode'] = config['agent_params'].get('evaluation_mode', 'mode')
config['training_params']['update_log_interval'] = 1

ppo = config['agent_params'].get('ppo_params', {})
env = config.get('environment_params', {})
dd = env.get('drawdown_constraint', {})
tp = config.get('training_params', {})

print('CONFIG SNAPSHOT')
print('Phase:', config['phase_name'])
print('Tickers:', config['ASSET_TICKERS'])
print('Date range:', config['ANALYSIS_START_DATE'], '->', config['ANALYSIS_END_DATE'])
print('Architecture:', config['agent_params']['actor_critic_type'])
print('TCN filters:', config['agent_params'].get('tcn_filters'))
print('Dirichlet activation:', config['agent_params'].get('dirichlet_alpha_activation'))
print('PPO: epochs=', ppo.get('num_ppo_epochs'), 'clip=', ppo.get('policy_clip'), 'actor_lr=', ppo.get('actor_lr'), 'critic_lr=', ppo.get('critic_lr'), 'target_kl=', ppo.get('target_kl'))

print()
print('REWARD + RISK CONTROLS')
print('target_turnover=', env.get('target_turnover'), 'turnover_penalty_scalar=', env.get('turnover_penalty_scalar'))
print('concentration_penalty_scalar=', env.get('concentration_penalty_scalar'), 'top_weight_penalty_scalar=', env.get('top_weight_penalty_scalar'))
print('action_realization_penalty_scalar=', env.get('action_realization_penalty_scalar'))
print('penalty_budget_ratio=', env.get('penalty_budget_ratio'))
print('drawdown: penalty_coef=', dd.get('penalty_coef'), 'lambda_floor=', dd.get('lambda_floor'), 'lambda_carry_decay=', dd.get('lambda_carry_decay'))
print('drawdown: target=', dd.get('target'), 'tolerance=', dd.get('tolerance'), 'lambda_max=', dd.get('lambda_max'))

print()
print('INTRA-STEP TAPE DELTA')
print('enabled=', env.get('intra_step_tape_delta_enabled'), 'window=', env.get('intra_step_tape_delta_window'), 'min_history=', env.get('intra_step_tape_delta_min_history'))
print('beta=', env.get('intra_step_tape_delta_beta'), 'clip=', env.get('intra_step_tape_delta_clip'))

print()
print('CHECKPOINT RULES')
print('tape_checkpoint_threshold=', tp.get('tape_checkpoint_threshold'))
print('periodic_checkpoint_every_steps=', tp.get('periodic_checkpoint_every_steps'))
print('high_watermark_checkpoint_enabled=', tp.get('high_watermark_checkpoint_enabled'), 'threshold=', tp.get('high_watermark_sharpe_threshold'))
print('step_sharpe_checkpoint_enabled=', tp.get('step_sharpe_checkpoint_enabled'), 'threshold=', tp.get('step_sharpe_checkpoint_threshold'))

print()
print('CURRICULUM')
print('episode_length_curriculum_schedule=', tp.get('episode_length_curriculum_schedule'))
print('turnover_penalty_curriculum=', tp.get('turnover_penalty_curriculum'))


Forced notebook date window: 2003-09-02 -> 2025-09-01
Forced analysis window: 2003-09-02 -> 2025-09-01
Forced train split end: 2021-09-01
CONFIG SNAPSHOT
Phase: Phase1_Baseline_PPO
Tickers: ['MSFT', 'GOOGL', 'JPM', 'JNJ', 'XOM', 'PG', 'NEE', 'LIN', 'CAT', 'UNH']
Date range: 2003-09-02 -> 2025-09-01
Architecture: TCN
TCN filters: [64, 128, 128]
Dirichlet activation: elu
PPO: epochs= 4 clip= 0.1 actor_lr= 2e-05 critic_lr= 0.0003 target_kl= 0.015

REWARD + RISK CONTROLS
target_turnover= 0.6 turnover_penalty_scalar= 2.0
concentration_penalty_scalar= 2.0 top_weight_penalty_scalar= 1.5
action_realization_penalty_scalar= 0.5
penalty_budget_ratio= 1.25
drawdown: penalty_coef= 1.5 lambda_floor= 0.0 lambda_carry_decay= 0.7
drawdown: target= 0.18 tolerance= -0.015 lambda_max= 5.0

INTRA-STEP TAPE DELTA
enabled= True window= 60 min_history= 20
beta= 0.01 clip= 0.2

CHECKPOINT RULES
tape_checkpoint_threshold= 4.0
periodic_checkpoint_every_steps= 10000
high_watermark_checkpoint_enabled= True thresho

In [ ]:
# ============================================================================
# OPTIONAL OVERRIDES (SAFE, CURRENT-ALIGNED)
# ============================================================================
APPLY_OVERRIDES = False

if APPLY_OVERRIDES:
    # Compact run controls
    config['training_params']['max_total_timesteps'] = 30_000
    config['training_params']['timesteps_per_ppo_update'] = 504

    # PPO stability controls
    ppo = config['agent_params']['ppo_params']
    ppo.update({
        'policy_clip': 0.10,
        'num_ppo_epochs': 4,
        'actor_lr': 2e-5,
        'critic_lr': 3e-4,
        'target_kl': 0.015,
        'kl_stop_multiplier': 1.2,
        'minibatches_before_kl_stop': 1,
    })

    # Full-horizon episodes (disable episode-length curriculum)
    config['training_params']['use_episode_length_curriculum'] = False
    config['training_params']['episode_length_curriculum_schedule'] = [
        {'threshold': 0, 'limit': None},
    ]

    # Turnover / reward controls
    env = config['environment_params']
    env['target_turnover'] = 0.60
    env['turnover_target_band'] = 0.20
    env['turnover_penalty_scalar'] = 1.5
    env['dsr_scalar'] = 2.0
    config['training_params']['evaluation_turnover_penalty_scalar'] = 1.5

    # Penalty rebalancing
    env['concentration_penalty_scalar'] = 2.0
    env['concentration_target_hhi'] = 0.14
    env['top_weight_penalty_scalar'] = 1.5
    env['target_top_weight'] = 0.22
    env['action_realization_penalty_scalar'] = 0.5
    env['penalty_budget_ratio'] = 1.25

    # Drawdown controller (gentler early adaptation)
    dd = env['drawdown_constraint']
    dd.update({
        'enabled': True,
        'target': 0.18,
        'penalty_coef': 1.5,
        'dual_learning_rate': 0.10,
        'lambda_init': 0.50,
        'lambda_floor': 0.00,
        'lambda_max': 5.0,
        'lambda_carry_decay': 0.7,
        'tolerance': -0.015,
        'penalty_reference': 'trigger_boundary',
        'cooling_rate': 0.35,
    })

    # Intra-step TAPE delta shaping (reduced)
    env['intra_step_tape_delta_enabled'] = True
    env['intra_step_tape_delta_window'] = 60
    env['intra_step_tape_delta_min_history'] = 20
    env['intra_step_tape_delta_beta'] = 0.01
    env['intra_step_tape_delta_clip'] = 0.20

    # Step-level Sharpe checkpointing (save any step where Sharpe >= 0.5)
    config['training_params']['step_sharpe_checkpoint_enabled'] = True
    config['training_params']['step_sharpe_checkpoint_threshold'] = 0.5

    print('Overrides applied')
else:
    print('APPLY_OVERRIDES=False (using config defaults)')




In [10]:
# ============================================================================
# VARIANT SETTINGS (TCN FAMILY)
# ============================================================================
VARIANT_SETTINGS = {
    'TCN': {
        'actor_critic_type': 'TCN',
        'use_attention': False,
        'use_fusion': False,
        'results_root': Path('tcn_results'),
    },
    'TCN_ATTENTION': {
        'actor_critic_type': 'TCN_ATTENTION',
        'use_attention': True,
        'use_fusion': False,
        'results_root': Path('tcn_att_results'),
    },
    'TCN_FUSION': {
        'actor_critic_type': 'TCN_FUSION',
        'use_attention': False,
        'use_fusion': True,
        'results_root': Path('tcn_fusion_results'),
    },
}

ACTIVE_VARIANT = 'TCN_FUSION'  # change to: TCN, TCN_ATTENTION, TCN_FUSION

if ACTIVE_VARIANT not in VARIANT_SETTINGS:
    raise ValueError(f'Unsupported ACTIVE_VARIANT: {ACTIVE_VARIANT}')

v = VARIANT_SETTINGS[ACTIVE_VARIANT]
config['agent_params']['actor_critic_type'] = v['actor_critic_type']
config['agent_params']['use_attention'] = v['use_attention']
config['agent_params']['use_fusion'] = v['use_fusion']

config['training_params']['use_episode_length_curriculum'] = False
config['training_params']['episode_length_curriculum_schedule'] = [
    {'threshold': 0, 'limit': None},
]


LATEST_VARIANT = ACTIVE_VARIANT
LATEST_RESULTS_ROOT = str(v['results_root'])

print('✅ Variant applied:', ACTIVE_VARIANT)
print('results_root:', LATEST_RESULTS_ROOT)


✅ Variant applied: TCN_FUSION
results_root: tcn_fusion_results


## 3) Data Pipeline (Features + Actuarial)


In [11]:
config = get_active_config("phase1")

# Optional guard for legacy/invalid macro codes in old configs.
# Current config is already updated, so this is usually a no-op.
macro_cfg = config["feature_params"]["macro_data"]
invalid_legacy_codes = {"NAPM", "MOVEINDEX", "ISM/MAN_PMI", "MOVE"}
before = len(macro_cfg.get("fred_series_config", []))
macro_cfg["fred_series_config"] = [
    s for s in macro_cfg.get("fred_series_config", [])
    if s.get("code") not in invalid_legacy_codes
]
after = len(macro_cfg.get("fred_series_config", []))
print(f"FRED series config: {before} -> {after} (removed legacy invalid codes: {before - after})")

# Keep legacy engineered-column disables for backward-compatible runs.
legacy_macro_columns = {"ISM_MAN_PMI_level", "ISM_MAN_PMI_diff", "MOVE_level", "MOVE_zscore"}
disabled = set(config["feature_params"]["feature_selection"]["disabled_features"])
newly_added = sorted(list(legacy_macro_columns - disabled))
disabled.update(legacy_macro_columns)
config["feature_params"]["feature_selection"]["disabled_features"] = sorted(disabled)
if newly_added:
    print("Added legacy macro disables:", newly_added)
else:
    print("Legacy macro disable set already present")


FRED series config: 19 -> 19 (removed legacy invalid codes: 0)
Legacy macro disable set already present


In [12]:
# ============================================================================
# FEATURE PRUNING (APPLIED BEFORE NORMALIZATION)
# ============================================================================
fs = config.setdefault('feature_params', {}).setdefault('feature_selection', {})
fs['disable_features'] = True

redundant_features = [
    # Price-level / highly collinear technicals
    'EMA_12', 'EMA_26', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'SMA_50',
    'DMP_14', 'DMN_14', 'ATRr_14', 'NATR_14', 'VOL_SMA_20',

    # Cross-sectional overlaps
    'BetaRank', 'VolatilityRank', 'InverseVolRank',
    'CrossSectional_ZScore_LogReturn_1d',

    # Macro level duplicates (prefer diff/zscore)
    'EFFR_level', 'SOFR_level', 'FEDFUNDS_level',
    'DGS10_level', 'DGS2_level', 'T10Y2Y_level',
    'TIPS10Y_level', 'BreakevenInf10Y_level', 'BreakevenInf5Y_level',
    'IG_Credit_level', 'HY_Credit_level',
]

existing_disabled = set(fs.get('disabled_features', []))
fs['disabled_features'] = sorted(existing_disabled.union(redundant_features))

print('Feature pruning enabled:', fs.get('disable_features', False))
print('Disabled feature count:', len(fs['disabled_features']))
print('Sample disabled features:', fs['disabled_features'][:25])


Feature pruning enabled: True
Disabled feature count: 65
Sample disabled features: ['ATRr_14', 'BAMLC0A0CMEY_diff', 'BAMLC0A0CMEY_level', 'BAMLC0A0CMEY_zscore', 'BAMLH0A0HYM2_diff', 'BAMLH0A0HYM2_level', 'BAMLH0A0HYM2_zscore', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BetaRank', 'BreakevenInf10Y_level', 'BreakevenInf5Y_level', 'CPI_level', 'CPI_mom', 'CPI_yoy', 'CrossSectional_ZScore_LogReturn_1d', 'DAAA_level', 'DAAA_zscore', 'DGS10_level', 'DGS2_level', 'DMN_14', 'DMP_14', 'EFFR_level', 'EMA_12']


In [13]:
# ============================================================================
# DATA LOADING + FEATURE ENGINEERING
# ============================================================================
processor = DataProcessor(config)

print('=' * 80)
print('LOADING AND PROCESSING DATA')
print('=' * 80)

FORCE_DATA_REFRESH = True  # Set True once when you want to rebuild cache for new date bounds
raw_df = processor.load_ohlcv_data(
    start_date=config['DATA_FETCH_START_DATE'],
    end_date=config['DATA_FETCH_END_DATE'],
    force_download=FORCE_DATA_REFRESH,
)
print('Raw shape:', raw_df.shape)
print('Raw dates:', raw_df['Date'].min(), '→', raw_df['Date'].max())

# Core feature pipeline
df = processor.calculate_log_returns(raw_df, periods=[1, 5, 10, 21])
df = processor.calculate_return_statistics(df, window=21)
df = processor.calculate_technical_indicators(df)
df = processor.calculate_dynamic_covariance_features(df)
df = processor.add_regime_features(df)
df = processor.add_fundamental_features(df)

macro_cfg = config.get('feature_params', {}).get('macro_data')
if macro_cfg is not None:
    macro_df, macro_cols = processor._build_macro_feature_frame(macro_cfg, df['Date'].min(), df['Date'].max())
    if macro_df is not None and macro_cols:
        df = df.merge(macro_df, on='Date', how='left')
        print(f'Macro features added: {len(macro_cols)}')

df = processor.add_quant_alpha_features(df)
df = processor.add_cross_sectional_features(df)
df = processor.add_actuarial_features(df)

master_df = df.copy()
feature_cols = processor.get_feature_columns('phase1')
present_feature_cols = [c for c in feature_cols if c in master_df.columns]

print('Final master_df shape:', master_df.shape)
print('Expected feature cols:', len(feature_cols))
print('Present feature cols :', len(present_feature_cols))

nan_counts = master_df[present_feature_cols].isna().sum()
nan_cols = nan_counts[nan_counts > 0].sort_values(ascending=False)
print('Feature columns with NaN:', len(nan_cols))
if len(nan_cols) > 0:
    display(nan_cols.head(20))


LOADING AND PROCESSING DATA
Raw shape: (55107, 7)
Raw dates: 2003-09-02 00:00:00 → 2025-08-29 00:00:00
Macro features added: 43
Final master_df shape: (54897, 118)
Expected feature cols: 66
Present feature cols : 66
Feature columns with NaN: 11


,0
MACDs_12_26_9,330
MACDh_12_26_9,330
ADX_14,270
MACD_12_26_9,250
RealizedSkew_21d,200
RollingVolatility_21d,200
DownsideSemiVar_21d,200
RealizedKurtosis_21d,200
STOCHd_14_3_3,170
RSI_14,140


In [14]:
master_df.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'LogReturn_1d', 'LogReturn_5d', 'LogReturn_10d',
       ...
       'LogReturn_1d_ZScore', 'RollingVolatility_21d_ZScore', 'RSI_14_ZScore',
       'BetaRank', 'VolatilityRank', 'InverseVolRank',
       'Actuarial_Expected_Recovery', 'Actuarial_Prob_30d',
       'Actuarial_Prob_60d', 'Actuarial_Reserve_Severity'],
      dtype='object', length=118)

In [15]:
act_cols = [c for c in master_df.columns if c.startswith("Actuarial_")]
print("Actuarial columns present:", len(act_cols))
print(act_cols[:10])

if act_cols:
    print(master_df[act_cols].describe().T[["mean","std","min","max"]].head(20))

Actuarial columns present: 4
['Actuarial_Expected_Recovery', 'Actuarial_Prob_30d', 'Actuarial_Prob_60d', 'Actuarial_Reserve_Severity']
                                  mean        std  min    max
Actuarial_Expected_Recovery  42.012050  76.714815  0.0  554.5
Actuarial_Prob_30d            0.606652   0.369684  0.0    1.0
Actuarial_Prob_60d            0.670421   0.338598  0.0    1.0
Actuarial_Reserve_Severity    0.718121   0.334307  0.0    1.0


In [16]:
# ============================================================================
# FIXED DATE SPLIT (TRAIN/TEST)
# ============================================================================
analysis_start = pd.Timestamp(config.get('ANALYSIS_START_DATE', '2003-09-02'))
analysis_end = pd.Timestamp(config.get('ANALYSIS_END_DATE', '2025-09-01'))

train_end_date = pd.Timestamp(config.get('TRAIN_TEST_SPLIT_DATE', '2021-09-01'))
test_start_date = train_end_date + pd.Timedelta(days=1)
test_end_date = analysis_end

all_dates = pd.to_datetime(master_df['Date'])
master_df = master_df[(all_dates >= analysis_start) & (all_dates <= analysis_end)].copy()
all_dates = pd.to_datetime(master_df['Date'])

train_mask = all_dates <= train_end_date
test_mask = (all_dates >= test_start_date) & (all_dates <= test_end_date)

train_df = master_df[train_mask].copy()
test_df = master_df[test_mask].copy()

print('Analysis range:', analysis_start.date(), '→', analysis_end.date())
print('Train split end:', train_end_date.date(), '| Test start:', test_start_date.date())
print('Train:', train_df['Date'].min(), '→', train_df['Date'].max(), f'({len(train_df):,} rows)')
print('Test :', test_df['Date'].min(), '→', test_df['Date'].max(), f'({len(test_df):,} rows)')





Analysis range: 2003-09-02 → 2025-09-01
Train split end: 2021-09-01 | Test start: 2021-09-02
Train: 2003-10-01 00:00:00 → 2021-09-01 00:00:00 (44,877 rows)
Test : 2021-09-02 00:00:00 → 2025-08-29 00:00:00 (10,020 rows)


In [17]:
train_df.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'LogReturn_1d', 'LogReturn_5d', 'LogReturn_10d',
       ...
       'LogReturn_1d_ZScore', 'RollingVolatility_21d_ZScore', 'RSI_14_ZScore',
       'BetaRank', 'VolatilityRank', 'InverseVolRank',
       'Actuarial_Expected_Recovery', 'Actuarial_Prob_30d',
       'Actuarial_Prob_60d', 'Actuarial_Reserve_Severity'],
      dtype='object', length=118)

In [18]:
# ============================================================================
# NORMALIZATION (TRAIN-FIT, TEST-TRANSFORM)
# ============================================================================
from collections import Counter

feature_cols = processor.get_feature_columns('phase1')

master_df_normalized, scalers = processor.normalize_features(
    master_df,
    feature_cols=feature_cols,
    train_end_date=train_end_date,
    test_start_date=test_start_date,
    existing_scalers=None,
    scaler_type='standard',
)

actuarial_cols = [c for c in master_df_normalized.columns if c.startswith('Actuarial_')]

method_counter = Counter()
for c in feature_cols:
    spec = scalers.get(c)
    if isinstance(spec, dict):
        method_counter[str(spec.get('method', 'standard'))] += 1
    elif spec is None:
        method_counter['missing'] += 1
    else:
        method_counter['legacy_scaler'] += 1

print('✅ Normalization complete')
print('Normalized shape:', master_df_normalized.shape)
print('Actuarial columns:', actuarial_cols)
print('Normalization strategy counts:', dict(method_counter))


✅ Normalization complete
Normalized shape: (54897, 118)
Actuarial columns: ['Actuarial_Expected_Recovery', 'Actuarial_Prob_30d', 'Actuarial_Prob_60d', 'Actuarial_Reserve_Severity']
Normalization strategy counts: {'robust_winsor': 34, 'bounded': 8, 'standard': 24}


In [19]:
# ============================================================================
# BUILD PHASE1 DATASET CONTAINER
# ============================================================================
all_dates_norm = pd.to_datetime(master_df_normalized['Date'])
train_df_norm = master_df_normalized[all_dates_norm <= train_end_date].copy()
test_df_norm = master_df_normalized[(all_dates_norm >= test_start_date) & (all_dates_norm <= test_end_date)].copy()

covariance_columns = identify_covariance_columns(master_df_normalized.columns)

phase1_data = Phase1Dataset(
    master_df=master_df_normalized,
    train_df=train_df_norm,
    test_df=test_df_norm,
    scalers=scalers,
    train_end_date=train_end_date,
    test_start_date=test_start_date,
    covariance_columns=covariance_columns,
    data_processor=processor,
)

print('✅ Phase1Dataset ready')
print('Train shape:', phase1_data.train_df.shape)
print('Test shape :', phase1_data.test_df.shape)
print('Covariance features:', len(covariance_columns))
print('Assets:', sorted(phase1_data.master_df['Ticker'].dropna().unique().tolist()))


✅ Phase1Dataset ready
Train shape: (44877, 118)
Test shape : (10020, 118)
Covariance features: 3
Assets: ['CAT', 'GOOGL', 'JNJ', 'JPM', 'LIN', 'MSFT', 'NEE', 'PG', 'UNH', 'XOM']


In [20]:
active_cols = [c for c in phase1_data.data_processor.get_feature_columns("phase1") if c in phase1_data.master_df.columns]
core_cols = [c for c in ["Date","Ticker","Open","High","Low","Close","Volume"] if c in phase1_data.master_df.columns]
keep = core_cols + active_cols

phase1_data.master_df = phase1_data.master_df[keep].copy()
phase1_data.train_df = phase1_data.train_df[keep].copy()
phase1_data.test_df  = phase1_data.test_df[keep].copy()
print(phase1_data.train_df.shape, phase1_data.test_df.shape)

(44877, 73) (10020, 73)


In [21]:
phase1_data.train_df.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'LogReturn_1d', 'LogReturn_5d', 'LogReturn_10d', 'LogReturn_21d',
       'RollingVolatility_21d', 'DownsideSemiVar_21d', 'RealizedSkew_21d',
       'RealizedKurtosis_21d', 'MACD_12_26_9', 'MACDh_12_26_9',
       'MACDs_12_26_9', 'RSI_14', 'STOCHd_14_3_3', 'ADX_14', 'OBV', 'MFI_14',
       'Covariance_Eigenvalue_0', 'Covariance_Eigenvalue_1',
       'Covariance_Eigenvalue_2', 'Fundamental_FCFE_Delta',
       'Fundamental_Revenue_Delta', 'Fundamental_NCFO_Delta',
       'Fundamental_FCFE_Sign', 'Fundamental_Staleness_Days',
       'Fundamental_Staleness_Quarters', 'Regime_Volatility_Ratio',
       'Regime_Price_vs_SMA_Short', 'Regime_SMA_Short_Slope',
       'Regime_SMA_Long_Slope', 'Regime_Momentum_Short',
       'Regime_Momentum_Long', 'Regime_Breadth_Positive',
       'Regime_Corr_to_Market', 'Residual_Momentum_21', 'Volume_Percentile_63',
       'YieldCurve_Spread', 'YieldCurve_Inverted_Flag', 'ShortTerm_Revers

In [22]:
feature_cols = set(phase1_data.data_processor.get_feature_columns("phase1"))
print({"Open","High","Low","Close","Volume"} & feature_cols)  # should be empty

set()


In [23]:
feature_cols = phase1_data.data_processor.get_feature_columns("phase1")
macro_cols = [c for c in feature_cols if c.startswith((
    "EFFR_", "SOFR_", "FEDFUNDS_", "DGS", "T10Y2Y_", "TIPS", "Breakeven", "IG_Credit_", "HY_Credit_", "YieldCurve_"
))]
print("macro feature count:", len(macro_cols))
print(macro_cols)

macro feature count: 17
['YieldCurve_Spread', 'YieldCurve_Inverted_Flag', 'EFFR_diff', 'EFFR_zscore', 'SOFR_diff', 'FEDFUNDS_diff', 'FEDFUNDS_zscore', 'DGS10_diff', 'DGS10_slope', 'DGS2_diff', 'TIPS10Y_diff', 'BreakevenInf10Y_diff', 'BreakevenInf5Y_diff', 'IG_Credit_diff', 'IG_Credit_zscore', 'HY_Credit_diff', 'HY_Credit_zscore']


In [24]:
disabled = set(config["feature_params"]["feature_selection"]["disabled_features"])
active = set(phase1_data.data_processor.get_feature_columns("phase1"))

print("Disabled ∩ Active:", sorted(disabled & active))
print("Count:", len(disabled & active))  # should be 0

Disabled ∩ Active: []
Count: 0


In [25]:
# PRE-TRAIN ACTIVE FEATURE CHECK
disabled = set(config["feature_params"]["feature_selection"]["disabled_features"])
active = set(phase1_data.data_processor.get_feature_columns("phase1"))

overlap = sorted(disabled & active)
print("disable_features:", config["feature_params"]["feature_selection"]["disable_features"])
print("disabled count:", len(disabled))
print("active count:", len(active))
print("disabled ∩ active:", len(overlap))
print("overlap sample:", overlap[:20])

assert config["feature_params"]["feature_selection"]["disable_features"] is True, "disable_features is not enabled"
assert len(overlap) == 0, f"Some disabled features are still active: {overlap[:10]}"
print("✅ Pre-train check passed: pruned features are not in active feature list.")

disable_features: True
disabled count: 65
active count: 66
disabled ∩ active: 0
overlap sample: []
✅ Pre-train check passed: pruned features are not in active feature list.


## 4) Training


In [ ]:
# Optional stability tweaks (keep commented unless needed)
# config['training_params']['timesteps_per_ppo_update'] = 504
# ppo = config['agent_params']['ppo_params']
# ppo['batch_size_ppo'] = 512
# ppo['num_ppo_epochs'] = 5
# ppo['actor_lr'] = 5e-5
# ppo['critic_lr'] = 1e-4
# ppo['policy_clip'] = 0.15
# ppo['target_kl'] = 0.02
# ppo['entropy_coef'] = 0.01
# config['environment_params']['penalty_budget_ratio'] = 1.25
# config['environment_params']['intra_step_tape_delta_beta'] = 0.10
# config['environment_params']['intra_step_tape_delta_clip'] = 0.20


In [ ]:
ap = config["agent_params"]
ppo = ap["ppo_params"]
tp = config["training_params"]
ep = config["environment_params"]

# PPO stability
ppo["actor_lr"] = 1e-5
ppo["critic_lr"] = 2e-4
ppo["num_ppo_epochs"] = 2
ppo["batch_size_ppo"] = 128
ppo["target_kl"] = 0.02
ppo["entropy_coef"] = 0.003
ppo["policy_clip"] = 0.08

# rollout/update cadence
tp["timesteps_per_ppo_update"] = 256

# model size (reduce volatility + OOM risk)
ap["sequence_length"] = 40
ap["tcn_filters"] = [32, 64, 64]

# reward noise reduction + turnover discipline
ep["intra_step_tape_delta_enabled"] = False
ep["dsr_scalar"] = 1.0
ep["target_turnover"] = 0.50
ep["turnover_penalty_scalar"] = 1.75


### Optuna Tuning

##### Run 1

In [ ]:
ap = config["agent_params"]
ppo = ap["ppo_params"]
tp = config["training_params"]
ep = config["environment_params"]

# PPO stability
ppo["actor_lr"] = 1e-5
ppo["critic_lr"] = 2e-4
ppo["num_ppo_epochs"] = 1
ppo["policy_clip"] = 0.06
ppo["target_kl"] = 0.02
ppo["entropy_coef"] = 0.002
ppo["max_grad_norm"] = 0.3

# short tuning budget
tp["timesteps_per_ppo_update"] = 256
tp["max_total_timesteps"] = 20000

config["training_params"]["use_episode_length_curriculum"] = True
config["training_params"]["episode_length_curriculum_schedule"] = [
    {"threshold": 0, "limit": 252},
    {"threshold": 10000, "limit": 504},
    {"threshold": 30000, "limit": 1000},
    {"threshold": 60000, "limit": None},
]

# reduce reward noise, enforce turnover discipline
ep["intra_step_tape_delta_enabled"] = False
ep["dsr_scalar"] = 1.0
ep["target_turnover"] = 0.50
ep["turnover_penalty_scalar"] = 2.0

##### Run 2

In [26]:
# ============================================================================
# NEXT RUN TUNING OVERRIDES (stability-first + risk-aux + rollout schedule)
# ============================================================================
from copy import deepcopy

config = deepcopy(config)

tp = config["training_params"]
ppo = config["agent_params"]["ppo_params"]
env = config["environment_params"]

# 1) Train length + PPO rollout schedule
tp["max_total_timesteps"] = 100_000

# Fallback defaults (used if schedule is missing)
tp["timesteps_per_ppo_update"] = 384
tp["batch_size_ppo"] = 96

# Rollout schedule (less noisy updates than starting at 256)
tp["timesteps_per_ppo_update_schedule"] = [
    {"threshold": 0, "timesteps_per_update": 384},
    {"threshold": 30_000, "timesteps_per_update": 512},
]

# Batch-size schedule (roughly rollout / 4)
tp["batch_size_ppo_schedule"] = [
    {"threshold": 0, "batch_size": 96},
    {"threshold": 30_000, "batch_size": 128},
]

# 2) PPO stability tightening
ppo["num_ppo_epochs"] = 1
ppo["actor_lr"] = 1.0e-5
ppo["critic_lr"] = 1.2e-4
ppo["target_kl"] = 0.012
ppo["kl_stop_multiplier"] = 1.2
ppo["entropy_coef"] = 0.0015
ppo["max_grad_norm"] = 0.20
ppo["policy_clip"] = 0.05

tp["actor_lr_schedule"] = [
    {"threshold": 0, "lr": 1.0e-5},
    {"threshold": 35_000, "lr": 8.0e-6},
    {"threshold": 70_000, "lr": 6.0e-6},
]

# 3) Episode horizon curriculum
tp["use_episode_length_curriculum"] = True
tp["episode_length_curriculum_schedule"] = [
    {"threshold": 0, "limit": 252},
    {"threshold": 10_000, "limit": 504},
    {"threshold": 25_000, "limit": 756},
    {"threshold": 40_000, "limit": None},
]

tp["log_step_diagnostics"] = True

# 4) Turnover control (steady but not overly punitive)
tp["turnover_penalty_curriculum"] = {
    0: 1.5,
    10_000: 2.0,
    25_000: 2.5,
    40_000: 3.0,
}
tp["evaluation_turnover_penalty_scalar"] = 3.0

env["target_turnover"] = 0.35
env["turnover_penalty_scalar"] = 1.5
env["transaction_cost_pct"] = 0.0005
env["action_realization_penalty_scalar"] = 1.0

# 5) Signed terminal reward + tighter Gate A
env["tape_terminal_bonus_mode"] = "signed"
env["tape_terminal_baseline"] = 0.20
env["tape_terminal_scalar"] = 6.0
env["tape_terminal_clip"] = 6.0
env["tape_terminal_gate_a_enabled"] = True
env["tape_terminal_gate_a_sharpe_threshold"] = 0.0
env["tape_terminal_gate_a_max_drawdown"] = 0.23
env["tape_terminal_neutral_band_halfwidth"] = 0.003

# 6) Risk-aware actor auxiliary objective (moderate strength)
ppo["use_risk_aux_loss"] = True
ppo["risk_aux_return_feature_index"] = 0
ppo["risk_aux_cash_return"] = 0.0
ppo["risk_aux_sharpe_coef"] = 0.03
ppo["risk_aux_mvo_coef"] = 0.001
ppo["risk_aux_mvo_cov_ridge"] = 1e-3
ppo["risk_aux_mvo_long_only"] = True
ppo["risk_aux_mvo_risky_budget"] = 0.95

ap = config["agent_params"]

# Keep stable activation
ap["dirichlet_alpha_activation"] = "elu"

# Less noisy sampling than current {max:0.5, min:0.1}
ap["dirichlet_epsilon"] = {"max": 0.35, "min": 0.20}

# Flatten logits a bit to reduce extreme weight jumps
ap["dirichlet_logit_temperature"] = 1.25

# Prevent overconfident alpha spikes
ap["dirichlet_alpha_cap"] = 40.0

# Keep as-is unless you switch activation to exp_clip
ap["dirichlet_exp_clip"] = (-5.0, 3.0)

env["action_execution_beta"] = 0.20
tp["action_execution_beta_curriculum"] = {
    0: 0.20,
    30_000: 0.35,
}
tp["evaluation_action_execution_beta"] = 0.35



print("✅ Applied next-run stability-first tuning")
print("max_total_timesteps:", tp["max_total_timesteps"])
print("timesteps_per_ppo_update_schedule:", tp["timesteps_per_ppo_update_schedule"])
print("batch_size_ppo_schedule:", tp["batch_size_ppo_schedule"])
print("target_kl:", ppo["target_kl"], "| kl_stop_multiplier:", ppo["kl_stop_multiplier"], "| policy_clip:", ppo["policy_clip"])
print("actor_lr:", ppo["actor_lr"], "| critic_lr:", ppo["critic_lr"])
print("turnover_penalty_curriculum:", tp["turnover_penalty_curriculum"])
print("terminal gate mdd:", env["tape_terminal_gate_a_max_drawdown"])
print("risk_aux:", ppo["use_risk_aux_loss"], "| sharpe_coef:", ppo["risk_aux_sharpe_coef"], "| mvo_coef:", ppo["risk_aux_mvo_coef"])
print("Dirichlet:", ap["dirichlet_alpha_activation"], ap["dirichlet_epsilon"],
      "| temp:", ap["dirichlet_logit_temperature"], "| alpha_cap:", ap["dirichlet_alpha_cap"])

✅ Applied next-run stability-first tuning
max_total_timesteps: 100000
timesteps_per_ppo_update_schedule: [{'threshold': 0, 'timesteps_per_update': 384}, {'threshold': 30000, 'timesteps_per_update': 512}]
batch_size_ppo_schedule: [{'threshold': 0, 'batch_size': 96}, {'threshold': 30000, 'batch_size': 128}]
target_kl: 0.012 | kl_stop_multiplier: 1.2 | policy_clip: 0.05
actor_lr: 1e-05 | critic_lr: 0.00012
turnover_penalty_curriculum: {0: 1.5, 10000: 2.0, 25000: 2.5, 40000: 3.0}
terminal gate mdd: 0.23
risk_aux: True | sharpe_coef: 0.03 | mvo_coef: 0.001
Dirichlet: elu {'max': 0.35, 'min': 0.2} | temp: 1.25 | alpha_cap: 40.0


In [27]:
# ============================================================================
# TRAIN ACTIVE VARIANT
# ============================================================================
RUN_TRAINING = True

if RUN_TRAINING:
    train_cfg = config['training_params']
    print('🚀 Starting training')
    print('Variant:', ACTIVE_VARIANT)
    print('max_total_timesteps:', train_cfg['max_total_timesteps'])
    print('timesteps_per_ppo_update:', train_cfg['timesteps_per_ppo_update'])

    experiment6 = run_experiment6_tape(
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        timesteps_per_update=train_cfg['timesteps_per_ppo_update'],
        max_total_timesteps=train_cfg['max_total_timesteps'],
    )

    print('✅ Training complete')
    print('checkpoint_prefix:', experiment6.checkpoint_path)
else:
    print('ℹ️ RUN_TRAINING=False (set True to train)')


🚀 Starting training
Variant: TCN_FUSION
max_total_timesteps: 100000
timesteps_per_ppo_update: 384

EXPERIMENT 6: TCN_FUSION Enhanced + TAPE Three-Component
Architecture: TCN + Fusion
Results root: /content/adaptive_portfolio_rl/tcn_fusion_results
Working dir: /content/adaptive_portfolio_rl
Covariance Features: Yes
🎯 REWARD SYSTEM: TAPE (Three-Component v3)
   Profile: BalancedGrowth
   Daily: Base + DSR/PBRS + Turnover_Proximity
   Terminal: mode=signed | baseline=0.20 | scalar=6.0 (clipped ±6.0)
   Gate A: enabled (Sharpe ≤ 0.00 or MDD ≥ 23.0% -> force non-positive terminal bonus)
   Neutral Band: enabled (±0.003 around baseline)
   🔄 Profile Manager: disabled (static profile only)
🎲 Experiment Seed: 6042 (Base: 42, Offset: 6000)
✅ Features: Enhanced (includes 3 covariance eigenvalues)
   Eigenvalues: ['Covariance_Eigenvalue_0', 'Covariance_Eigenvalue_1', 'Covariance_Eigenvalue_2']
   Train shape: (44877, 73)
   Test shape: (10020, 73)

🏗️ Creating THREE-COMPONENT TAPE v3 environments

🔄 Update 1/216 | Step 384/100,000 | Episode 1 | Time: 66.6s
   📊 Metrics: Return=+21.33% | Sharpe=0.849 | DD=23.45% | Turnover=11.42%
   🎚️ Intra-Step TAPE: potential=0.7406 | delta_reward=+0.0000
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0418 | critic_loss=0.6106 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.3053 | risk_aux_total=-0.0038 | sharpe_proxy=0.1266 | sharpe_loss=-0.0038 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000010 | critic_lr=0.000120 | target_kl=0.0120 | rollout=384 | batch_size=96
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 0.44% / trig 16.50%) | terminal=0.000 (peak 0.500) | TAPE=0.4147
   🎯 Episode 2: TAPE Score = 0.5335 (bonus: +2.50 → +2.50)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00002_shp1p029_actor.weights.h5 (Sharpe=1.029)
   🎯 Episode 3: TAPE Score = 0.6747 (bonus: +3.56 → +3.56)
      💾 Sharpe-threshold checkpoint saved: /content/adaptiv

🔄 Update 2/216 | Step 768/100,000 | Episode 3 | Time: 123.8s
   📊 Metrics: Return=+23.32% | Sharpe=1.841 | DD=4.43% | Turnover=10.80%
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0353 | critic_loss=0.3020 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1510 | risk_aux_total=-0.0001 | sharpe_proxy=0.0028 | sharpe_loss=-0.0001 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000010 | critic_lr=0.000120 | target_kl=0.0120 | rollout=384 | batch_size=96
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 0.13% / trig 16.50%) | terminal=0.000 (peak 0.000) | TAPE=0.6747
   🎯 Episode 4: TAPE Score = 0.2654 (bonus: -0.49 → -0.49)
      🚦 Gate A applied: Sharpe=0.539, MDD=31.03%
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00004_shp0p539_actor.weights.h5 (Sharpe=0.539)


🔄 Update 3/216 | Step 1,152/100,000 | Episode 4 | Time: 178.5s
   📊 Metrics: Return=+15.27% | Sharpe=0.539 | DD=31.03% | Turnover=11.13%
   🎚️ Intra-Step TAPE: potential=0.5203 | delta_reward=-0.0003
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0349 | critic_loss=0.8223 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.4111 | risk_aux_total=-0.0041 | sharpe_proxy=0.1373 | sharpe_loss=-0.0041 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000010 | critic_lr=0.000120 | target_kl=0.0120 | rollout=384 | batch_size=96
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 1.48% / trig 16.50%) | terminal=0.000 (peak 0.113) | TAPE=0.2654
   🎯 Episode 5: TAPE Score = 0.6310 (bonus: +3.23 → +3.23)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00005_shp1p631_actor.weights.h5 (Sharpe=1.631)
      Rare checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/rare_models/exp6_tape_ep5_sh1.631_dd

🔄 Update 4/216 | Step 1,536/100,000 | Episode 6 | Time: 233.9s
   📊 Metrics: Return=+27.23% | Sharpe=3.396 | DD=2.23% | Turnover=10.86%
   🎚️ Intra-Step TAPE: potential=0.7396 | delta_reward=-0.0000
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=-0.0538 | critic_loss=0.3626 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1813 | risk_aux_total=-0.0044 | sharpe_proxy=0.1459 | sharpe_loss=-0.0044 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000010 | critic_lr=0.000120 | target_kl=0.0120 | rollout=384 | batch_size=96
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 0.71% / trig 16.50%) | terminal=0.000 (peak 0.000) | TAPE=0.7520
   🎯 Episode 7: TAPE Score = 0.5240 (bonus: +2.43 → +2.43)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00007_shp0p953_actor.weights.h5 (Sharpe=0.953)


...

🔄 Update 211/216 | Step 97,920/100,000 | Episode 112 | Time: 14049.0s
   📊 Metrics: Return=+89.36% | Sharpe=1.182 | DD=12.75% | Turnover=20.32%
   🎚️ Intra-Step TAPE: potential=0.6869 | delta_reward=+0.0003
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0039 | critic_loss=0.2257 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1129 | risk_aux_total=-0.0002 | sharpe_proxy=0.0058 | sharpe_loss=-0.0002 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0120 | rollout=512 | batch_size=128
   🎯 Episode 113: TAPE Score = 0.3709 (bonus: -1.28 → -1.28)
      🚦 Gate A applied: Sharpe=0.874, MDD=33.16%
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00113_shp0p874_actor.weights.h5 (Sharpe=0.874)

📚 EPISODE HORIZON UPDATE at 98,432 steps:
   Episode horizon set to full dataset


🔄 Update 212/216 | Step 98,432/100,000 | Episode 113 | Time: 14121.1s
   📊 Metrics: Return=+147.52% | Sharpe=0.874 | DD=33.16% | Turnover=20.47%
   🎚️ Intra-Step TAPE: potential=0.6203 | delta_reward=+0.0002
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0986 | critic_loss=0.4188 | mean_adv=-0.0000
   🧮 Loss Detail: critic_scaled=0.2094 | risk_aux_total=-0.0035 | sharpe_proxy=0.1185 | sharpe_loss=-0.0036 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0120 | rollout=512 | batch_size=128
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 1.30% / trig 16.50%) | terminal=0.000 (peak 0.143) | TAPE=0.3709

📚 EPISODE HORIZON UPDATE at 98,944 steps:
   Episode horizon set to full dataset


🔄 Update 213/216 | Step 98,944/100,000 | Episode 113 | Time: 14193.8s
   📊 Metrics: Return=+32.56% | Sharpe=0.938 | DD=6.99% | Turnover=20.50%
   🎚️ Intra-Step TAPE: potential=0.6057 | delta_reward=+0.0026
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0107 | critic_loss=0.2435 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1217 | risk_aux_total=-0.0001 | sharpe_proxy=0.0050 | sharpe_loss=-0.0002 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0120 | rollout=512 | batch_size=128

📚 EPISODE HORIZON UPDATE at 99,456 steps:
   Episode horizon set to full dataset


🔄 Update 214/216 | Step 99,456/100,000 | Episode 113 | Time: 14266.1s
   📊 Metrics: Return=+81.33% | Sharpe=1.058 | DD=10.81% | Turnover=20.37%
   🎚️ Intra-Step TAPE: potential=0.7292 | delta_reward=+0.0000
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0337 | critic_loss=0.2252 | mean_adv=-0.0000
   🧮 Loss Detail: critic_scaled=0.1126 | risk_aux_total=0.0016 | sharpe_proxy=-0.0528 | sharpe_loss=0.0016 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0120 | rollout=512 | batch_size=128

📚 EPISODE HORIZON UPDATE at 99,968 steps:
   Episode horizon set to full dataset


🔄 Update 215/216 | Step 99,968/100,000 | Episode 113 | Time: 14338.5s
   📊 Metrics: Return=+119.10% | Sharpe=0.908 | DD=13.35% | Turnover=20.38%
   🎚️ Intra-Step TAPE: potential=0.5292 | delta_reward=+0.0023
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0543 | critic_loss=0.2628 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1314 | risk_aux_total=-0.0030 | sharpe_proxy=0.0992 | sharpe_loss=-0.0030 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0120 | rollout=512 | batch_size=128
      💾 Periodic checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/exp6_tape_step100000_actor.weights.h5

📚 EPISODE HORIZON UPDATE at 100,000 steps:
   Episode horizon set to full dataset
🔄 Update 216/216 | Step 100,000/100,000 | Episode 113 | Time: 14344.3s
   📊 Metrics: Return=+126.08% | Sharpe=0.927 | DD=13.35% | Turnover=20.40%
   🎚️ Intra-Step TAPE: potential=0.3802 | delta_reward=-0.0003
   🎯 Profile: BalancedGrowth
   🧠 Training: 

In [36]:
# list files inside results
!find ./tcn_fusion_results -maxdepth 3 -type f | head -n 100

./tcn_fusion_results/exp6_tape_step090000_actor.weights.h5
./tcn_fusion_results/exp6_tape_step010000_actor.weights.h5
./tcn_fusion_results/exp6_tape_ep32_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep27_sh2.266_dd5.6_critic.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep39_sh2.205_dd8.5_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep32_sh2.639_dd4.3_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep32_sh2.639_dd4.3_critic.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep24_sh2.128_dd8.1_critic.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep39_sh2.205_dd8.5_critic.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep27_sh2.266_dd5.6_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep6_sh3.396_dd2.2_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep24_sh2.128_dd8.1_actor.weights.h5
./tcn_fusion_results/rare_models/exp6_tape_ep6_sh3.396_dd2.2_critic.weights.h5
./tcn_fusion_results/exp6_tape_ep6_c

In [34]:
# Download /content/adaptive_portfolio_rl/tcn_fusion_results as a zip
from pathlib import Path
import shutil
from google.colab import files

src = Path("/content/adaptive_portfolio_rl/data")
zip_base = Path("/content/data1")

if not src.exists():
    raise FileNotFoundError(f"Not found: {src}")

zip_path = shutil.make_archive(str(zip_base), "zip", root_dir=str(src.parent), base_dir=src.name)
print("Created:", zip_path)

files.download(zip_path)

Created: /content/data1.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/data1.zip /content/drive/MyDrive/
print("Copied to Google Drive: MyDrive/data1.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copied to Google Drive: MyDrive/data1.zip


In [ ]:
# ============================================================================
# QUICK TRAINING LOG INSPECTION (LATEST)
# ============================================================================
logs_dir = Path(LATEST_RESULTS_ROOT) / 'logs'
logs_dir.mkdir(parents=True, exist_ok=True)

episodes_files = sorted(logs_dir.glob('*episodes*.csv'), key=lambda p: p.stat().st_mtime, reverse=True)
if not episodes_files:
    print(f'No episodes CSV found in {logs_dir} yet.')
else:
    epis_path = episodes_files[0]
    episodes_df = pd.read_csv(epis_path)
    print('Episodes file:', epis_path)
    print('Rows:', len(episodes_df))
    display(episodes_df.head())


## 5) Evaluation (Unified Multi-Track)


In [ ]:
# ============================================================================
# OPTIONAL: RELOAD TRAINING METADATA (POST-RESTART)
# ============================================================================
USE_METADATA_RELOAD = False
METADATA_PATH = None  # e.g., Path('tcn_results/logs/Exp6_TCN_Enhanced_TAPE_training_YYYYMMDD_HHMMSS_metadata.json')

if USE_METADATA_RELOAD:
    if METADATA_PATH is None:
        logs_dir = Path(LATEST_RESULTS_ROOT) / 'logs'
        cand = sorted(logs_dir.glob('*metadata*.json'), key=lambda p: p.stat().st_mtime, reverse=True)
        METADATA_PATH = cand[0] if cand else None

    if METADATA_PATH and Path(METADATA_PATH).exists():
        config = load_training_metadata_into_config(Path(METADATA_PATH), config, verbose=True)
        print('✅ Metadata reloaded from:', METADATA_PATH)
    else:
        print('⚠️ Metadata file not found; continuing with current config.')
else:
    print('ℹ️ USE_METADATA_RELOAD=False')


In [ ]:
# ============================================================================
# UNIFIED EVALUATION: det_mode + det_mean + stochastic
# ============================================================================
RUN_EVAL = True

# Model selection
MODEL_FAMILY = 'normal'           # normal | rare | clip
NORMAL_MODEL_STRATEGY = 'latest'  # latest | final
RARE_MODEL_STRATEGY = 'best'      # best | episode
CHECKPOINT_EPISODE = 54           # used when rare_model_strategy='episode'
CLIP_EPISODE = 54                 # used when model_family='clip'
CHECKPOINT_PREFIX_OVERRIDE = None # e.g., 'tcn_fusion_results/exp6_tape_ep83'

if RUN_EVAL:
    experiment6_stub = create_experiment6_result_stub(
        random_seed=RANDOM_SEED,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        checkpoint_path=None,
        base_agent_params=config.get('agent_params'),
    )

    evaluation_stub = evaluate_experiment6_checkpoint(
        experiment6_stub,
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        model_family=MODEL_FAMILY,
        normal_model_strategy=NORMAL_MODEL_STRATEGY,
        rare_model_strategy=RARE_MODEL_STRATEGY,
        checkpoint_episode=CHECKPOINT_EPISODE,
        clip_episode=CLIP_EPISODE,
        checkpoint_path_override=CHECKPOINT_PREFIX_OVERRIDE,
        num_eval_runs=10,
        compare_deterministic_modes=['mode', 'mean'],
        stochastic_eval_mode='sample',
        sample_actions_stochastic=True,
        sample_actions=None,
        stochastic_episode_length_limit=252,
        save_eval_logs=True,
        save_eval_artifacts=True,
    )

    print('✅ Evaluation complete')
    print('Checkpoint:', evaluation_stub.actor_weights_path)
    print('Eval CSV  :', evaluation_stub.eval_results_path)
else:
    print('ℹ️ RUN_EVAL=False (set True to evaluate)')


In [ ]:
# ============================================================================
# EVALUATION ARTIFACT EXPORTS (VARIANT-SCOPED)
# ============================================================================
from datetime import datetime

if 'evaluation_stub' not in globals():
    print('Run evaluation first (RUN_EVAL=True).')
else:
    assets = ASSET_TICKERS + ['Cash']

    results_root = Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    out_root = results_root / 'model_outputs' / f'eval_{stamp}'
    det_out = out_root / 'deterministic'
    sto_out = out_root / 'stochastic'
    det_out.mkdir(parents=True, exist_ok=True)
    sto_out.mkdir(parents=True, exist_ok=True)

    # Deterministic artifacts
    det_dates = pd.DatetimeIndex(evaluation_stub.env_test_deterministic.dates)

    if len(evaluation_stub.deterministic_alphas) > 0:
        df_alpha = pd.DataFrame(evaluation_stub.deterministic_alphas, columns=assets)
        df_alpha.index = det_dates[:len(df_alpha)]
        df_alpha.index.name = 'date'
        df_alpha.to_csv(det_out / 'alphas.csv')

    if len(evaluation_stub.deterministic_weights) > 0:
        df_w = pd.DataFrame(evaluation_stub.deterministic_weights, columns=assets)
        df_w.index = det_dates[:len(df_w)]
        df_w.index.name = 'date'
        df_w.to_csv(det_out / 'weights.csv')

    if len(evaluation_stub.deterministic_actions) > 0:
        df_a = pd.DataFrame(evaluation_stub.deterministic_actions, columns=assets)
        df_a.index = det_dates[:len(df_a)]
        df_a.index.name = 'date'
        df_a.to_csv(det_out / 'actions.csv')

    # Copy eval summary CSV into output root for traceability
    eval_csv_path = Path(evaluation_stub.eval_results_path) if evaluation_stub.eval_results_path else None
    if eval_csv_path and eval_csv_path.exists():
        df_eval = pd.read_csv(eval_csv_path)
        df_eval.to_csv(out_root / 'evaluation_summary.csv', index=False)
    else:
        df_eval = pd.DataFrame()

    # Stochastic artifacts
    all_dates = pd.DatetimeIndex(evaluation_stub.env_test_random.dates)
    actions_rows, weights_rows, alphas_rows = [], [], []

    if isinstance(evaluation_stub.stochastic_results, pd.DataFrame) and not evaluation_stub.stochastic_results.empty:
        stochastic_results_df = evaluation_stub.stochastic_results.copy()
        stochastic_results_df.to_csv(sto_out / 'stochastic_results.csv', index=False)

        for i in range(len(stochastic_results_df)):
            run_id = int(stochastic_results_df.iloc[i].get('run', i + 1))
            start_date = pd.Timestamp(stochastic_results_df.iloc[i]['start_date'])
            start_idx = all_dates.get_loc(start_date)

            run_actions = evaluation_stub.stochastic_actions[i] if i < len(evaluation_stub.stochastic_actions) else []
            run_weights = evaluation_stub.stochastic_weights[i] if i < len(evaluation_stub.stochastic_weights) else []
            run_alphas = evaluation_stub.stochastic_alphas[i] if i < len(evaluation_stub.stochastic_alphas) else []

            run_dates = all_dates[start_idx:start_idx + len(run_weights)]

            if len(run_actions):
                dfa = pd.DataFrame(run_actions, columns=assets)
                dfa['run'] = run_id
                dfa['date'] = run_dates[:len(dfa)]
                actions_rows.append(dfa)

            if len(run_weights):
                dfw = pd.DataFrame(run_weights, columns=assets)
                dfw['run'] = run_id
                dfw['date'] = run_dates[:len(dfw)]
                weights_rows.append(dfw)

            if len(run_alphas):
                dfl = pd.DataFrame(run_alphas, columns=assets)
                dfl['run'] = run_id
                dfl['date'] = run_dates[:len(dfl)]
                alphas_rows.append(dfl)

    if actions_rows:
        pd.concat(actions_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'actions_all_runs.csv')
    if weights_rows:
        pd.concat(weights_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'weights_all_runs.csv')
    if alphas_rows:
        pd.concat(alphas_rows, ignore_index=True).set_index(['run', 'date']).to_csv(sto_out / 'alphas_all_runs.csv')

    # README with current run context
    readme_lines = [
        '# Evaluation Artifact Export',
        '',
        f'- Variant results root: `{results_root}`',
        f'- Export root: `{out_root}`',
        f'- Checkpoint actor: `{evaluation_stub.actor_weights_path}`',
        f'- Checkpoint critic: `{evaluation_stub.critic_weights_path}`',
        f'- Eval summary CSV: `{evaluation_stub.eval_results_path}`',
        f'- Export timestamp: `{stamp}`',
        '',
        '## Included Files',
        '- `deterministic/weights.csv`',
        '- `deterministic/actions.csv`',
        '- `deterministic/alphas.csv`',
        '- `stochastic/stochastic_results.csv` (if stochastic runs were executed)',
        '- `stochastic/weights_all_runs.csv`',
        '- `stochastic/actions_all_runs.csv`',
        '- `stochastic/alphas_all_runs.csv`',
        '- `evaluation_summary.csv`',
    ]

    if not df_eval.empty:
        cols = [
            'eval_track', 'evaluation_type', 'start_date', 'market_regime',
            'mean_concentration_hhi', 'mean_top_weight',
            'mean_action_realization_l1', 'max_action_realization_l1'
        ]
        present = [c for c in cols if c in df_eval.columns]
        readme_lines += ['', '## Key Logged Diagnostics (present in summary CSV)', *(f'- `{c}`' for c in present)]

    (out_root / 'README.md').write_text('\n'.join(readme_lines), encoding='utf-8')

    print('✅ Export complete')
    print('Export root:', out_root)
    print('Deterministic dir:', det_out)
    print('Stochastic dir   :', sto_out)


In [ ]:
# ============================================================================
# EVAL CSV DIAGNOSTIC COLUMN CHECK
# ============================================================================
required_cols = [
    'start_date',
    'market_regime',
    'mean_concentration_hhi',
    'mean_top_weight',
    'mean_action_realization_l1',
    'max_action_realization_l1',
]

csv_path = None

if 'evaluation_stub' in globals() and getattr(evaluation_stub, 'eval_results_path', None):
    p = Path(evaluation_stub.eval_results_path)
    if p.exists():
        csv_path = p

if csv_path is None:
    root = Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    logs_dir = root / 'logs'
    candidates = sorted(logs_dir.glob('*_eval_*.csv'), key=lambda x: x.stat().st_mtime, reverse=True) if logs_dir.exists() else []
    csv_path = candidates[0] if candidates else None

if csv_path is None:
    print('⚠️ No evaluation CSV found. Run evaluation first.')
else:
    df_eval = pd.read_csv(csv_path)
    present = [c for c in required_cols if c in df_eval.columns]
    missing = [c for c in required_cols if c not in df_eval.columns]

    print('📂 Eval CSV:', csv_path)
    print('Rows:', len(df_eval))
    print('Required columns present:', len(present), '/', len(required_cols))

    if missing:
        print('❌ Missing columns:', missing)
    else:
        print('✅ All required diagnostic columns are present.')

    show_cols = ['eval_track', 'evaluation_type'] + [c for c in required_cols if c in df_eval.columns]
    show_cols = [c for c in show_cols if c in df_eval.columns]
    if show_cols:
        display(df_eval[show_cols].head(10))


In [ ]:
# ============================================================================
# DIAGNOSTICS SUMMARY
# ============================================================================
if 'evaluation_stub' not in globals():
    print('Run evaluation first (RUN_EVAL=True).')
else:
    # stochastic summary
    stoch = evaluation_stub.stochastic_results.copy()
    if stoch is not None and not stoch.empty:
        cols = [
            'total_return', 'annualized_return', 'sharpe_ratio', 'sortino_ratio',
            'max_drawdown', 'volatility', 'turnover', 'win_rate'
        ]
        cols = [c for c in cols if c in stoch.columns]
        print('Stochastic summary:')
        display(stoch[cols].describe().T)

    # deterministic diagnostics
    acts = np.asarray(evaluation_stub.deterministic_actions)
    alps = np.asarray(evaluation_stub.deterministic_alphas)

    action_uniques = int(np.unique(np.round(acts, 6), axis=0).shape[0]) if acts.size else 0
    alpha_le1_frac = float(np.mean(alps <= 1.0)) if alps.size else 0.0
    argmax_uniques = int(np.unique(np.argmax(alps, axis=1)).shape[0]) if (alps.ndim == 2 and len(alps) > 0) else 0

    print('Deterministic diagnostics:')
    print(' action_uniques      =', action_uniques)
    print(' alpha<=1 fraction   =', alpha_le1_frac)
    print(' argmax_alpha_uniques=', argmax_uniques)


In [ ]:
# ============================================================================
# FIXED STRESS-WINDOW EVALUATION (2 WINDOWS)
# ============================================================================
import pandas as pd
from dataclasses import replace

STRESS_WINDOWS = [
    ('2020-02-20', '2020-05-29', 'COVID crash + rebound'),
    ('2022-01-03', '2022-12-30', 'Rate-hike bear year'),
]

def subset_phase1_test_window(phase1_data, start_date, end_date):
    s = pd.Timestamp(start_date)
    e = pd.Timestamp(end_date)
    df = phase1_data.test_df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    win = df[(df['Date'] >= s) & (df['Date'] <= e)].copy()
    if win.empty:
        raise ValueError(f'No rows in test_df for {start_date} -> {end_date}')
    return replace(
        phase1_data,
        test_df=win,
        test_start_date=win['Date'].min(),
        train_end_date=win['Date'].max(),
    )

fixed_rows = []
for start, end, label in STRESS_WINDOWS:
    phase_win = subset_phase1_test_window(phase1_data, start, end)

    ev = evaluate_experiment6_checkpoint(
        experiment6_stub,
        phase1_data=phase_win,
        config=config,
        random_seed=RANDOM_SEED,
        checkpoint_path_override=CHECKPOINT_PREFIX_OVERRIDE,
        deterministic_eval_mode='mode',
        num_eval_runs=0,
        stochastic_eval_mode='sample',
        save_eval_logs=False,
        save_eval_artifacts=False,
    )

    m = ev.deterministic_metrics or {}
    fixed_rows.append({
        'window_label': label,
        'start': start,
        'end': end,
        'days_traded': len(ev.deterministic_portfolio) - 1 if len(ev.deterministic_portfolio) else 0,
        'total_return': m.get('total_return'),
        'annualized_return': m.get('annualized_return'),
        'sharpe': m.get('sharpe_ratio'),
        'sortino': m.get('sortino_ratio'),
        'max_drawdown': m.get('max_drawdown_abs', m.get('max_drawdown')),
        'volatility': m.get('volatility'),
        'turnover': m.get('turnover'),
        'win_rate': m.get('win_rate'),
    })

fixed_df = pd.DataFrame(fixed_rows)
display(fixed_df.sort_values('start'))


## 6) Checkpoint Scan


In [ ]:
import os, socket
print("cwd:", os.getcwd())
print("hostname:", socket.gethostname())
print("exists:", os.path.exists("/content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260219_210119_metadata.json"))

In [ ]:
from copy import deepcopy
from pathlib import Path

meta = Path("tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260219_210119_metadata.json")
config = load_training_metadata_into_config(meta, deepcopy(config), verbose=True)

# hard enforce fusion
config["agent_params"]["actor_critic_type"] = "TCN_FUSION"
config["agent_params"]["use_fusion"] = True
config["agent_params"]["use_attention"] = False

In [ ]:
# ============================================================================
# RUN CHECKPOINT SCAN (LOCAL + ROBUST)
# ============================================================================
from pathlib import Path
import re
import pandas as pd

def resolve_results_root():
    candidates = [
        # Local Windows path (your machine)
        #Path(r"C:\Users\Owner\new_project\adaptive_portfolio_rl\tcn_fusion_results"),
        # WSL path to same folder
        #Path("/mnt/c/Users/Owner/new_project/adaptive_portfolio_rl/tcn_fusion_results"),
        # Relative local fallback
        #Path("./tcn_fusion_results"),

        # Colab fallbacks (kept just in case)
        Path("/content/adaptive_portfolio_rl/tcn_fusion_results/rare_models"),
        Path("/content/adaptive_portfolio_rl/tcn_results"),
        Path("/content/adaptive_portfolio_rl/tcn_fusion_results00"),
        Path("/content/adaptive_portfolio_rl/tcn_results00"),
    ]
    existing = [p for p in candidates if p.exists()]
    if not existing:
        raise FileNotFoundError("No results root found (local or /content).")
    # Prefer local by candidate order (first match)
    return existing[0]

def evaluate_checkpoint_range_deterministic_all(
    episode_range=(8, 100),
    results_root=None,
    random_seed=RANDOM_SEED,
    deterministic_eval_mode="mean",
):
    low, high = episode_range
    base_root = Path(results_root) if results_root else resolve_results_root()

    actor_paths = sorted(base_root.rglob("*_actor.weights.h5"))
    print(f"Found actor files (all subdirs): {len(actor_paths)} under {base_root}")

    rows = []
    skipped = 0

    for actor_path in actor_paths:
        m_ep = re.search(r"_ep(\d+)", actor_path.name)
        if not m_ep:
            continue
        ep = int(m_ep.group(1))
        if not (low <= ep <= high):
            continue

        prefix = str(actor_path).replace("_actor.weights.h5", "")
        ckpt_group = actor_path.parent.name

        try:
            stub = create_experiment6_result_stub(
                random_seed=RANDOM_SEED,
                use_covariance=True,
                architecture=arch,
                checkpoint_path=prefix,
                agent_config=agent_cfg,      # important
                base_agent_params=None,      # avoid default drift
            )


            ev = evaluate_experiment6_checkpoint(
                experiment6=stub,
                phase1_data=phase1_data,
                config=config,
                random_seed=random_seed,
                checkpoint_path_override=prefix,
                model_family="normal",
                normal_model_strategy="latest",
                num_eval_runs=0,
                deterministic_eval_mode=deterministic_eval_mode,
                save_eval_logs=False,
                save_eval_artifacts=False,
            )

            m = ev.deterministic_metrics or {}
            rows.append({
                "episode": ep,
                "source_dir": ckpt_group,
                "checkpoint_prefix": prefix,
                "sharpe": m.get("sharpe_ratio", float("nan")),
                "total_return": m.get("total_return", float("nan")),
                "max_drawdown": m.get("max_drawdown_abs", m.get("max_drawdown", float("nan"))),
                "turnover": m.get("turnover", float("nan")),
            })
        except Exception as e:
            skipped += 1
            print(f"Skipping incompatible checkpoint: {actor_path.name} | {type(e).__name__}: {e}")

    if not rows:
        print(f"No checkpoints in range {episode_range} under {base_root}")
        return None

    df_scores = pd.DataFrame(rows).sort_values("sharpe", ascending=False).reset_index(drop=True)
    print(f"Evaluated: {len(df_scores)} | Skipped: {skipped}")
    return df_scores

# Run
RUN_SCAN = True

if RUN_SCAN:
    results_root = resolve_results_root()
    print("Scanning results root:", results_root)

    scan_df = evaluate_checkpoint_range_deterministic_all(
        episode_range=(1, 300),
        results_root=results_root,
        random_seed=RANDOM_SEED,
        deterministic_eval_mode="mean",
    )

    display(scan_df.head(20) if scan_df is not None else None)

    step_dir = results_root / "step_sharpe_checkpoints"
    if step_dir.exists():
        step_files = sorted(step_dir.glob("*_actor.weights.h5"))
        print(f"Step-Sharpe checkpoints found: {len(step_files)}")
        for p in step_files[:10]:
            print(" -", p.name)
    else:
        print("No step_sharpe_checkpoints directory yet.")
else:
    print("RUN_SCAN=False")

In [ ]:
from copy import deepcopy
from pathlib import Path
import json

results_root = Path("tcn_fusion_results")
meta = results_root / "logs" / "Exp6_TCN_FUSION_Enhanced_TAPE_training_20260219_210119_metadata.json"

# 1) align config to run
config = load_training_metadata_into_config(meta, deepcopy(config), verbose=True)

with open(meta, "r", encoding="utf-8") as f:
    md = json.load(f)

agent_cfg = md["Architecture_Settings"]["agent_params_effective"]
arch = md["Architecture_Settings"]["resolved_architecture"]
meta_mtime = meta.stat().st_mtime

# 2) filter actor checkpoints to this run window (newer than metadata)
all_actor_paths = sorted(results_root.rglob("*_actor.weights.h5"))
actor_paths = [p for p in all_actor_paths if p.stat().st_mtime >= meta_mtime - 5]

print("all:", len(all_actor_paths), "filtered:", len(actor_paths))

In [ ]:
# ============================================================================
# CHECKPOINT SCANNER (DETERMINISTIC)
# ============================================================================
import re


def evaluate_checkpoint_range_deterministic(
    episode_range=(2, 300),
    results_root=None,
    random_seed=RANDOM_SEED,
    deterministic_eval_mode='mode',
):
    low, high = episode_range
    base_root = Path(results_root) if results_root else Path(LATEST_RESULTS_ROOT)

    checkpoints = {}
    for root in [base_root, base_root / 'rare_models']:
        if not root.exists():
            continue
        for actor_path in root.glob('*_actor.weights.h5'):
            m = re.search(r'_ep(\d+)', actor_path.name)
            if not m:
                continue
            ep = int(m.group(1))
            if low <= ep <= high:
                checkpoints[ep] = actor_path

    if not checkpoints:
        print(f'No checkpoints found in {base_root} for range {episode_range}.')
        return None

    rows = []
    for ep, actor_path in sorted(checkpoints.items()):
        prefix = str(actor_path).replace('_actor.weights.h5', '')

        stub = create_experiment6_result_stub(
            random_seed=random_seed,
            use_covariance=True,
            architecture=config['agent_params']['actor_critic_type'],
            checkpoint_path=prefix,
            base_agent_params=config.get('agent_params'),
        )

        ev = evaluate_experiment6_checkpoint(
            experiment6=stub,
            phase1_data=phase1_data,
            config=config,
            random_seed=random_seed,
            checkpoint_path_override=prefix,
            model_family='normal',
            normal_model_strategy='latest',
            num_eval_runs=0,
            deterministic_eval_mode=deterministic_eval_mode,
            save_eval_logs=False,
            save_eval_artifacts=False,
        )

        m = ev.deterministic_metrics or {}
        rows.append({
            'episode': ep,
            'checkpoint_prefix': prefix,
            'sharpe': m.get('sharpe_ratio', float('nan')),
            'total_return': m.get('total_return', float('nan')),
            'max_drawdown': m.get('max_drawdown_abs', m.get('max_drawdown', float('nan'))),
            'turnover': m.get('turnover', float('nan')),
        })

    df_scores = pd.DataFrame(rows).sort_values('sharpe', ascending=False)
    return df_scores


In [ ]:
results_root

In [ ]:
# ============================================================================
# RUN CHECKPOINT SCAN
# ============================================================================
RUN_SCAN = True

if RUN_SCAN:
    results_root = Path("/content/adaptive_portfolio_rl") #Path(globals().get('LATEST_RESULTS_ROOT', 'tcn_results'))
    print('Scanning:', results_root)
    scan_df = evaluate_checkpoint_range_deterministic(
        episode_range=(8, 100),
        results_root=results_root,
        random_seed=RANDOM_SEED,
        deterministic_eval_mode='mean',
    )
    display(scan_df.head(20) if scan_df is not None else None)

    step_dir = results_root / 'step_sharpe_checkpoints'
    if step_dir.exists():
        step_files = sorted(step_dir.glob('*_actor.weights.h5'))
        print(f'Step-Sharpe checkpoints found: {len(step_files)}')
        for p in step_files[:10]:
            print(' -', p.name)
    else:
        print('No step_sharpe_checkpoints directory yet.')
else:
    print('ℹ️ RUN_SCAN=False')


## 7) Overfit Monitor (Train-Test Gap)


In [37]:
# ============================================================================
# OVERFIT MONITOR HELPERS (VM-safe + scan all checkpoint folders)
# ============================================================================
import re
from dataclasses import replace
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd


def _infer_results_root_for_notebook(cfg):
    arch = cfg.get("agent_params", {}).get("actor_critic_type", "TCN").upper()
    use_attention = bool(cfg.get("agent_params", {}).get("use_attention", False))
    use_fusion = bool(cfg.get("agent_params", {}).get("use_fusion", False))

    if arch.startswith("TCN"):
        if use_fusion:
            rel = "tcn_fusion_results"
        elif use_attention:
            rel = "tcn_att_results"
        else:
            rel = "tcn_results"
    else:
        rel = "tcn_results"

    candidates = [
        Path("/content/adaptive_portfolio_rl") / rel,  # Colab VM
        Path(rel),                                     # local cwd fallback
    ]
    for c in candidates:
        if c.exists():
            return c
    return candidates[0]


def _maybe_load_metadata_into_config(config, results_root, metadata_path=None, auto_latest=True):
    cfg = config
    chosen = None

    if metadata_path:
        chosen = Path(metadata_path)
    elif auto_latest:
        log_dir = Path(results_root) / "logs"
        metas = sorted(log_dir.glob("*_metadata.json"), key=lambda p: p.stat().st_mtime)
        if metas:
            chosen = metas[-1]

    if chosen is None:
        return cfg, None

    if "load_training_metadata_into_config" not in globals():
        print(f"⚠️ metadata loader not found in scope, skipping metadata load: {chosen}")
        return cfg, None

    cfg = load_training_metadata_into_config(chosen, cfg, verbose=True)
    return cfg, chosen


def _discover_checkpoint_prefixes(
    results_root,
    episode_range=(1, 9999),
    include_root=True,
    include_rare=True,
    include_high_watermark=True,
    include_step_sharpe=True,
    include_other_subdirs=False,
    recursive=True,
    require_critic_pair=True,
):
    results_root = Path(results_root)
    lo, hi = episode_range
    actor_files = list(results_root.rglob("*_actor.weights.h5") if recursive else results_root.glob("*_actor.weights.h5"))

    ckpts = []
    for actor in actor_files:
        parent = actor.parent
        parent_name = parent.name
        is_root = parent.resolve() == results_root.resolve()

        allowed = False
        if is_root and include_root:
            allowed = True
        elif parent_name == "rare_models" and include_rare:
            allowed = True
        elif parent_name == "high_watermark_checkpoints" and include_high_watermark:
            allowed = True
        elif parent_name == "step_sharpe_checkpoints" and include_step_sharpe:
            allowed = True
        elif include_other_subdirs:
            allowed = True

        if not allowed:
            continue

        prefix = str(actor).replace("_actor.weights.h5", "")
        critic = Path(prefix + "_critic.weights.h5")
        if require_critic_pair and not critic.exists():
            continue

        name = actor.name
        m_ep = re.search(r"_ep(\d+)", name)
        m_step = re.search(r"_step(\d+)", name)

        ep = int(m_ep.group(1)) if m_ep else None
        step = int(m_step.group(1)) if m_step else None

        # Apply episode filter only to episode-tagged checkpoints.
        if ep is not None and not (lo <= ep <= hi):
            continue

        if "rare_models" in str(parent):
            kind = "rare"
        elif "high_watermark_checkpoints" in str(parent):
            kind = "high_watermark"
        elif "step_sharpe_checkpoints" in str(parent):
            kind = "step_sharpe"
        elif step is not None:
            kind = "periodic_step"
        else:
            kind = "root"

        ckpts.append({
            "checkpoint_prefix": prefix,
            "source_dir": str(parent),
            "checkpoint_kind": kind,
            "episode": ep,
            "step": step,
            "actor_path": str(actor),
            "critic_path": str(critic),
        })

    # De-dupe by prefix
    uniq = {}
    for c in ckpts:
        uniq[c["checkpoint_prefix"]] = c

    ckpts = list(uniq.values())
    ckpts.sort(key=lambda x: (
        x["episode"] if x["episode"] is not None else 10**9,
        x["step"] if x["step"] is not None else 10**12,
        x["checkpoint_prefix"],
    ))
    return ckpts


def _subset_phase1_for_eval(phase1_data, split="test"):
    split = str(split).lower().strip()
    if split not in {"train", "test"}:
        raise ValueError(f"split must be train or test, got: {split}")

    eval_df = phase1_data.train_df.copy() if split == "train" else phase1_data.test_df.copy()
    start_date = pd.to_datetime(eval_df["Date"]).min()
    end_date = pd.to_datetime(eval_df["Date"]).max()

    return replace(
        phase1_data,
        test_df=eval_df,
        test_start_date=start_date,
        train_end_date=end_date,
    )


def _diagnostics_from_eval(ev):
    acts = np.asarray(ev.deterministic_actions)
    alps = np.asarray(ev.deterministic_alphas)
    action_uniques = int(np.unique(np.round(acts, 6), axis=0).shape[0]) if acts.size else 0
    alpha_le1_fraction = float(np.mean(alps <= 1.0)) if alps.size else 0.0
    argmax_alpha_uniques = int(np.unique(np.argmax(alps, axis=1)).shape[0]) if (alps.ndim == 2 and len(alps) > 0) else 0
    return action_uniques, alpha_le1_fraction, argmax_alpha_uniques


def run_checkpoint_overfit_monitor(
    phase1_data,
    config,
    random_seed,
    episode_range=(1, 300),
    deterministic_modes=("mode", "mean"),
    eval_splits=("train", "test"),
    results_root=None,
    include_root=True,
    include_rare=True,
    include_high_watermark=True,
    include_step_sharpe=True,
    include_other_subdirs=False,
    recursive=True,
    require_critic_pair=True,
    metadata_path=None,
    auto_load_latest_metadata=True,
    save_csv=True,
):
    cfg = config.copy()
    results_root = Path(results_root) if results_root else _infer_results_root_for_notebook(cfg)
    if not results_root.exists():
        raise RuntimeError(f"results_root does not exist: {results_root}")

    cfg, used_meta = _maybe_load_metadata_into_config(
        cfg, results_root, metadata_path=metadata_path, auto_latest=auto_load_latest_metadata
    )
    if used_meta:
        print("📄 Using metadata:", used_meta)

    ckpts = _discover_checkpoint_prefixes(
        results_root,
        episode_range=episode_range,
        include_root=include_root,
        include_rare=include_rare,
        include_high_watermark=include_high_watermark,
        include_step_sharpe=include_step_sharpe,
        include_other_subdirs=include_other_subdirs,
        recursive=recursive,
        require_critic_pair=require_critic_pair,
    )
    if not ckpts:
        raise RuntimeError(f"No checkpoints found under {results_root} with current filters.")

    print(f"🔎 Discovered checkpoints: {len(ckpts)}")
    by_kind = pd.Series([c["checkpoint_kind"] for c in ckpts]).value_counts().to_dict()
    print("   by kind:", by_kind)

    if isinstance(deterministic_modes, str):
        deterministic_modes = (deterministic_modes,)
    if isinstance(eval_splits, str):
        eval_splits = (eval_splits,)
    eval_splits = tuple(str(s).lower().strip() for s in eval_splits)

    bad = [s for s in eval_splits if s not in {"train", "test"}]
    if bad:
        raise ValueError(f"Invalid eval_splits entries: {bad}. Allowed: train, test")

    stub = create_experiment6_result_stub(
        random_seed=random_seed,
        use_covariance=True,
        architecture=cfg["agent_params"]["actor_critic_type"],
        checkpoint_path=ckpts[0]["checkpoint_prefix"],
        base_agent_params=cfg.get("agent_params"),
    )

    rows = []
    for ck in ckpts:
        prefix = ck["checkpoint_prefix"]
        ep = ck["episode"]
        step = ck["step"]

        for split in eval_splits:
            phase_eval = _subset_phase1_for_eval(phase1_data, split=split)
            split_start = pd.to_datetime(phase_eval.test_df["Date"]).min()
            split_end = pd.to_datetime(phase_eval.test_df["Date"]).max()

            for mode in deterministic_modes:
                try:
                    ev = evaluate_experiment6_checkpoint(
                        stub,
                        phase1_data=phase_eval,
                        config=cfg,
                        random_seed=random_seed,
                        checkpoint_path_override=prefix,
                        deterministic_eval_mode=mode,
                        num_eval_runs=0,
                        stochastic_eval_mode="sample",
                        save_eval_logs=False,
                        save_eval_artifacts=False,
                    )
                    m = ev.deterministic_metrics or {}
                    action_uniques, alpha_le1_fraction, argmax_alpha_uniques = _diagnostics_from_eval(ev)

                    rows.append({
                        "checkpoint_prefix": prefix,
                        "checkpoint_kind": ck["checkpoint_kind"],
                        "source_dir": ck["source_dir"],
                        "episode": ep,
                        "step": step,
                        "architecture": cfg["agent_params"]["actor_critic_type"],
                        "split": split,
                        "deterministic_mode": mode,
                        "seed": random_seed,
                        "window_start": split_start,
                        "window_end": split_end,
                        "days_traded": int(len(ev.deterministic_portfolio) - 1) if len(ev.deterministic_portfolio) else 0,
                        "total_return": float(m.get("total_return", np.nan)),
                        "annualized_return": float(m.get("annualized_return", np.nan)),
                        "sharpe_ratio": float(m.get("sharpe_ratio", np.nan)),
                        "sortino_ratio": float(m.get("sortino_ratio", np.nan)),
                        "max_drawdown": float(m.get("max_drawdown_abs", m.get("max_drawdown", np.nan))),
                        "volatility": float(m.get("volatility", np.nan)),
                        "turnover": float(m.get("turnover", np.nan)),
                        "win_rate": float(m.get("win_rate", np.nan)),
                        "action_uniques": action_uniques,
                        "alpha_le1_fraction": alpha_le1_fraction,
                        "argmax_alpha_uniques": argmax_alpha_uniques,
                        "eval_error": "",
                    })
                except Exception as e:
                    rows.append({
                        "checkpoint_prefix": prefix,
                        "checkpoint_kind": ck["checkpoint_kind"],
                        "source_dir": ck["source_dir"],
                        "episode": ep,
                        "step": step,
                        "architecture": cfg["agent_params"]["actor_critic_type"],
                        "split": split,
                        "deterministic_mode": mode,
                        "seed": random_seed,
                        "window_start": split_start,
                        "window_end": split_end,
                        "days_traded": 0,
                        "total_return": np.nan,
                        "annualized_return": np.nan,
                        "sharpe_ratio": np.nan,
                        "sortino_ratio": np.nan,
                        "max_drawdown": np.nan,
                        "volatility": np.nan,
                        "turnover": np.nan,
                        "win_rate": np.nan,
                        "action_uniques": 0,
                        "alpha_le1_fraction": np.nan,
                        "argmax_alpha_uniques": 0,
                        "eval_error": str(e),
                    })

    df = pd.DataFrame(rows)
    if df.empty:
        raise RuntimeError("Monitor produced no rows.")

    ok = df[df["eval_error"] == ""].copy()
    left = ok[ok["split"] == "train"].copy()
    right = ok[ok["split"] == "test"].copy()

    if left.empty or right.empty:
        raise RuntimeError("Overfit summary requires BOTH successful train and test rows.")

    keys = [
        "checkpoint_prefix", "checkpoint_kind", "source_dir",
        "episode", "step", "architecture", "deterministic_mode", "seed"
    ]
    summary = left.merge(right, on=keys, suffixes=("_train", "_test"))

    summary["sharpe_gap"] = summary["sharpe_ratio_train"] - summary["sharpe_ratio_test"]
    summary["mdd_gap"] = summary["max_drawdown_test"] - summary["max_drawdown_train"]
    summary["return_gap"] = summary["annualized_return_train"] - summary["annualized_return_test"]

    summary["flag_overfit"] = (
        (summary["sharpe_gap"] > 0.40)
        | (summary["mdd_gap"] > 0.05)
        | (summary["return_gap"] > 0.10)
    )

    summary = summary.sort_values(
        ["flag_overfit", "sharpe_ratio_test"], ascending=[True, False]
    ).reset_index(drop=True)

    out_path = None
    if save_csv:
        out_dir = Path(results_root) / "logs"
        out_dir.mkdir(parents=True, exist_ok=True)
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = out_dir / f"checkpoint_overfit_monitor_{ts}.csv"
        summary.to_csv(out_path, index=False)
        print("💾 Overfit monitor saved:", out_path)

        err_path = out_dir / f"checkpoint_overfit_monitor_errors_{ts}.csv"
        df[df["eval_error"] != ""].to_csv(err_path, index=False)
        print("💾 Eval errors saved:", err_path)

    return df, summary, out_path


In [ ]:
results_root = Path("/content/adaptive_portfolio_rl/tcn_fusion_results")

df_all, df_summary, out_path = run_checkpoint_overfit_monitor(
    phase1_data=phase1_data,
    config=config,
    random_seed=RANDOM_SEED,
    episode_range=(100, 300),
    deterministic_modes=("mean",),          # keep stable
    eval_splits=("test"),
    results_root=results_root,
    include_root=True,
    include_rare=True,
    include_high_watermark=True,
    include_step_sharpe=True,               # set True even if empty
    include_other_subdirs=False,
    recursive=True,
    require_critic_pair=True,
    metadata_path=None,                     # auto-load latest metadata from logs
    auto_load_latest_metadata=True,
    save_csv=True,
)

display(df_summary.head(30))
print("rows all:", len(df_all), "rows summary:", len(df_summary))


✅ Applied training metadata to config
   Metadata: /content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260220_072945_metadata.json
   Run timestamp: 20260220_072945
   Architecture: TCN_FUSION
   Turnover target: 0.35
   DSR scalar: 2.0
   PPO update timesteps: scheduled
   Episode length curriculum: True
   Profile override loaded: True
   Credit assignment mode: step_reward_plus_terminal_bonus
   Retroactive episode scaling: False
📄 Using metadata: /content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260220_072945_metadata.json
🔎 Discovered checkpoints: 109
   by kind: {'high_watermark': 91, 'periodic_step': 10, 'rare': 5, 'root': 3}

LOADING CUSTOM CHECKPOINT: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00001_shp0p849
✅ Found actor weights: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00001_shp0p849_acto

   ✅ Models built successfully
📂 Loading checkpoint weights...

LOADING CUSTOM CHECKPOINT: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00004_shp0p539
✅ Found actor weights: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00004_shp0p539_actor.weights.h5
✅ Found critic weights: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00004_shp0p539_critic.weights.h5
🏗️ Recreating evaluation environments...


: 

In [ ]:
# ============================================================================
# RUN OVERFIT MONITOR
# ============================================================================
RUN_OVERFIT_MONITOR = True

if RUN_OVERFIT_MONITOR:
    results_root = Path(globals().get('LATEST_RESULTS_ROOT', _infer_results_root_for_notebook(config)))
    print('Using results root:', results_root)

    monitor_rows_df, monitor_summary_df, monitor_csv_path = run_checkpoint_overfit_monitor(
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        episode_range=(79, 100),
        deterministic_modes=('mode', 'mean'),
        eval_splits=('train', 'test'),
        results_root=results_root,
        include_rare=True,
        save_csv=True,
    )

    display(monitor_summary_df.head(20))

    if not monitor_summary_df.empty:
        best = (
            monitor_summary_df[monitor_summary_df['flag_overfit'] == False]
            .sort_values('sharpe_ratio_test', ascending=False)
            .head(10)
        )
        print('Top non-overfit candidates (by test Sharpe):')
        display(best[[
            'episode', 'deterministic_mode', 'sharpe_ratio_test',
            'max_drawdown_test', 'turnover_test', 'sharpe_gap', 'mdd_gap', 'return_gap'
        ]])
else:
    print('ℹ️ RUN_OVERFIT_MONITOR=False')


## 8) Optional Analysis Utilities


In [ ]:
# ============================================================================
# OPTIONAL: ABLATION TABLE + TRACK SUMMARY
# ============================================================================
RUN_OPTIONAL_ANALYSIS = False

if RUN_OPTIONAL_ANALYSIS:
    try:
        from src.notebook_helpers.tcn_phase1 import build_ablation_table, build_evaluation_track_summary
    except Exception as e:
        raise RuntimeError(f'Optional analysis helpers unavailable: {e}')

    available = {k: v for k, v in globals().items() if k.startswith('evaluation_') and hasattr(v, 'deterministic_metrics')}
    if 'evaluation_stub' in globals():
        available.setdefault('current_eval', evaluation_stub)

    if not available:
        print('No evaluation objects found. Run evaluation first.')
    else:
        display(build_ablation_table(available))
        if 'evaluation_stub' in globals():
            print('Track summary for current evaluation:')
            display(build_evaluation_track_summary(evaluation_stub))
else:
    print('ℹ️ RUN_OPTIONAL_ANALYSIS=False')


## 9) Run Checklist

Before running heavy jobs:
- Confirm `ACTIVE_VARIANT`
- Confirm `max_total_timesteps` and `timesteps_per_ppo_update`
- Confirm curriculum schedule (1500 -> 2000 -> 2500 -> full)
- Confirm step-Sharpe checkpoint rule (`>= 0.5`)
- Confirm intra-step TAPE delta settings (`beta`, `clip`, `window`)
- Set exactly one expensive toggle at a time (`RUN_TRAINING`, `RUN_EVAL`, `RUN_SCAN`, `RUN_OVERFIT_MONITOR`)
- Keep artifact exports on after successful eval


## 10) Feature Manifest Audit (Latest Artifacts)

Use this section to inspect the latest active-feature manifests without hardcoded timestamps.


In [ ]:
import json
from pathlib import Path
from src.notebook_helpers.tcn_phase1 import summarize_active_feature_manifest

def latest_path(paths):
    paths = [p for p in paths if p.exists()]
    if not paths:
        return None
    return sorted(paths, key=lambda p: p.stat().st_mtime, reverse=True)[0]

trainrl_manifest = latest_path(Path('results').glob('**/active_feature_manifest.json'))
notebook_manifest = latest_path(Path(LATEST_RESULTS_ROOT).glob('logs/*_active_feature_manifest.json'))

print('Latest train_rl manifest:', trainrl_manifest)
print('Latest notebook manifest:', notebook_manifest)

if trainrl_manifest:
    summarize_active_feature_manifest(str(trainrl_manifest))
if notebook_manifest:
    summarize_active_feature_manifest(str(notebook_manifest))


In [ ]:
# Inspect missing requested columns + group counts from latest notebook manifest
manifest_path = notebook_manifest
if manifest_path is None:
    print('No notebook manifest found. Run training first.')
else:
    m = json.loads(Path(manifest_path).read_text(encoding='utf-8'))
    train_env = m.get('train_env', m)
    print('Manifest path:', manifest_path)
    print('Missing requested columns:', train_env.get('missing_requested_columns', []))
    print('Group counts:', train_env.get('group_counts', {}))


In [ ]:
# Print active variable names from latest notebook manifest
manifest_path = notebook_manifest
if manifest_path is None:
    print('No notebook manifest found. Run training first.')
else:
    m = json.loads(Path(manifest_path).read_text(encoding='utf-8'))
    train_env = m.get('train_env', m)
    active = train_env.get('active_feature_columns', [])
    print(f'Active variable count: {len(active)}')
    for v in active:
        print(v)
